# 🧠  Economic Context & Synthesis (EDA + Modeling)
## 🎯 Mục tiêu
Phân tích các **biến kinh tế vĩ mô** ảnh hưởng đến việc **khách hàng có gửi tiền (y)** hay không.  
Sau đó, xây dựng các mô hình dự đoán với **PySpark**, xử lý dữ liệu **mất cân bằng** bằng **SMOTE**,  và đánh giá mô hình bằng **Cross-validation**.


In [43]:
# Import các thư viện cần thiết
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder



# Tạo SparkSession
spark = SparkSession.builder.appName("Hao_Economic_Context_Synthesis").getOrCreate()

# Đọc dữ liệu
data = spark.read.csv("../data/bank-additional/bank-additional-full.csv", header=True, sep=';', inferSchema=True)
print("Số dòng:", data.count())
data.printSchema()


Số dòng: 41188
root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- emp.var.rate: double (nullable = true)
 |-- cons.price.idx: double (nullable = true)
 |-- cons.conf.idx: double (nullable = true)
 |-- euribor3m: double (nullable = true)
 |-- nr.employed: double (nullable = true)
 |-- y: string (nullable = true)



In [44]:
data.show(5)

+---+---------+-------+-----------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
|age|      job|marital|  education|default|housing|loan|  contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp.var.rate|cons.price.idx|cons.conf.idx|euribor3m|nr.employed|  y|
+---+---------+-------+-----------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
| 56|housemaid|married|   basic.4y|     no|     no|  no|telephone|  may|        mon|     261|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|     5191.0| no|
| 57| services|married|high.school|unknown|     no|  no|telephone|  may|        mon|     149|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|     5191.0| no|
| 37| serv

## 🔍 2️⃣ EDA - Phân tích ảnh hưởng của các biến kinh tế vĩ mô
Ở đây ta tập trung vào 5 biến vĩ mô:
- `emp.var.rate` – tỷ lệ biến động việc làm  
- `cons.price.idx` – chỉ số giá tiêu dùng  
- `cons.conf.idx` – chỉ số niềm tin người tiêu dùng  
- `euribor3m` – lãi suất EURIBOR 3 tháng  
- `nr.employed` – số người có việc làm  

Ta sẽ xem xét **phân phối**, **tương quan** và **ảnh hưởng** của chúng tới biến mục tiêu `y`.


In [45]:
# Đổi tên cột để tránh lỗi có dấu chấm
data = (data
    .withColumnRenamed("emp.var.rate", "emp_var_rate")
    .withColumnRenamed("cons.price.idx", "cons_price_idx")
    .withColumnRenamed("cons.conf.idx", "cons_conf_idx")
    .withColumnRenamed("nr.employed", "nr_employed")
)

macro_cols = ["emp_var_rate", "cons_price_idx", "cons_conf_idx", "euribor3m", "nr_employed"]

In [46]:
from pyspark.sql.functions import col, mean, stddev, min as _min, max as _max, count, corr
from pyspark.sql.types import FloatType
# === ĐỔI TÊN CỘT CHO HỢP LỆ (nếu chưa làm) ===
data = (data
    .withColumnRenamed("emp.var.rate", "emp_var_rate")
    .withColumnRenamed("cons.price.idx", "cons_price_idx")
    .withColumnRenamed("cons.conf.idx", "cons_conf_idx")
    .withColumnRenamed("nr.employed", "nr_employed")
)

# Danh sách các biến vĩ mô
macro_cols = ["emp_var_rate", "cons_price_idx", "cons_conf_idx", "euribor3m", "nr_employed"]

# --- 1️⃣ Thống kê mô tả cơ bản ---
print("=== 1️⃣ Thống kê mô tả cơ bản ===")
data.select(macro_cols).describe().show(truncate=False)

# --- 2️⃣ Tỷ lệ các lớp mục tiêu (y) ---
print("=== 2️⃣ Tỷ lệ phân bố biến mục tiêu y ===")
total_count = data.count()
data.groupBy("y").agg(
    count("*").alias("count")
).withColumn(
    "percentage", (col("count") / total_count * 100)
).show()

# --- 3️⃣ Trung bình các biến vĩ mô theo từng giá trị y ---
print("=== 3️⃣ Trung bình các biến vĩ mô theo từng giá trị y ===")
agg_exprs = [mean(c).alias(f"avg_{c}") for c in macro_cols]
data.groupBy("y").agg(*agg_exprs).show(truncate=False)




=== 1️⃣ Thống kê mô tả cơ bản ===
+-------+-------------------+------------------+------------------+------------------+-----------------+
|summary|emp_var_rate       |cons_price_idx    |cons_conf_idx     |euribor3m         |nr_employed      |
+-------+-------------------+------------------+------------------+------------------+-----------------+
|count  |41188              |41188             |41188             |41188             |41188            |
|mean   |0.08188550063178392|93.57566436828918 |-40.50260027191787|3.6212908128585366|5167.035910944004|
|stddev |1.57095974051703   |0.5788400489541355|4.628197856174595 |1.7344474048512557|72.25152766825924|
|min    |-3.4               |92.201            |-50.8             |0.634             |4963.6           |
|max    |1.4                |94.767            |-26.9             |5.045             |5228.1           |
+-------+-------------------+------------------+------------------+------------------+-----------------+

=== 2️⃣ Tỷ lệ phân b

In [47]:
from pyspark.sql.functions import corr, when, col
import math

data = data.withColumn("y_num", when(col("y") == "yes", 1.0).when(col("y") == "no", 0.0).otherwise(col("y").cast("double")))

# --- 4️⃣ Ma trận tương quan giữa các biến vĩ mô và output ---
print("=== 4️⃣ Ma trận tương quan giữa các biến vĩ mô và output ===")

# Thêm biến y vào danh sách
all_cols = macro_cols + ["y_num"]

corr_matrix = []
for c1 in all_cols:
    row = []
    for c2 in all_cols:
        try:
            val = data.select(corr(c1, c2).alias("corr")).collect()[0]["corr"]
            if val is None or math.isnan(val):
                val = 0.0
        except Exception:
            val = 0.0
        row.append(float(val))
    corr_matrix.append((c1, *row))

# Tạo DataFrame hiển thị ma trận tương quan
columns = ["Feature"] + all_cols
corr_df = spark.createDataFrame(corr_matrix, columns)

corr_df.show(truncate=False)


=== 4️⃣ Ma trận tương quan giữa các biến vĩ mô và output ===
+--------------+-------------------+--------------------+--------------------+--------------------+-------------------+--------------------+
|Feature       |emp_var_rate       |cons_price_idx      |cons_conf_idx       |euribor3m           |nr_employed        |y_num               |
+--------------+-------------------+--------------------+--------------------+--------------------+-------------------+--------------------+
|emp_var_rate  |1.0                |0.775334170834832   |0.19604126813197284 |0.9722446711516147  |0.9069701012560353 |-0.29833442615937794|
|cons_price_idx|0.775334170834832  |1.0                 |0.058986181748833216|0.688230107037495   |0.5220339770130168 |-0.1362112128191817 |
|cons_conf_idx |0.19604126813197287|0.05898618174883324 |1.0                 |0.27768621966375506 |0.10051343183753894|0.05487794605319229 |
|euribor3m     |0.9722446711516147 |0.6882301070374951  |0.27768621966375506 |1.0            

- Nhóm biến emp.var.rate, euribor3m, nr.employed có thể được rút gọn hoặc chọn 1–2 đại diện.

- Các biến có tương quan âm với y

- Khi các chỉ số kinh tế này tăng (biểu hiện cho tình hình kinh tế tốt hơn), khả năng khách hàng đồng ý gửi tiền (y=1) lại giảm.

| Biến vĩ mô (`feature`) | Ý nghĩa kinh tế                 | Xu hướng khi biến tăng             | Ảnh hưởng đến khả năng gửi tiền (`y=1`) | Mức độ ảnh hưởng  | Ghi chú                                                                    |
| ---------------------- | ------------------------------- | ---------------------------------- | --------------------------------------- | ----------------- | -------------------------------------------------------------------------- |
| **emp.var.rate**       | Tỷ lệ biến động việc làm        | Kinh tế ổn định hơn, việc làm tăng | 🔻 Giảm khả năng gửi tiền               | **Mạnh (âm)**     | Khi tỷ lệ việc làm cao, người dân chi tiêu nhiều hơn, gửi tiết kiệm ít hơn |
| **cons.price.idx**     | Chỉ số giá tiêu dùng (lạm phát) | Lạm phát tăng                      | 🔻 Giảm nhẹ khả năng gửi tiền           | **Yếu (âm)**      | Ảnh hưởng nhỏ, thể hiện qua chênh lệch nhỏ giữa hai nhóm                   |
| **cons.conf.idx**      | Niềm tin người tiêu dùng        | Tâm lý tiêu dùng tích cực hơn      | ⚪ Gần như không ảnh hưởng               | **Rất yếu**       | Phân bố hai nhóm gần như trùng nhau                                        |
| **euribor3m**          | Lãi suất Euribor 3 tháng        | Lãi suất thị trường tăng           | 🔻 Giảm mạnh khả năng gửi tiền          | **Mạnh (âm)**     | Khi lãi suất cao, khách hàng ưu tiên đầu tư hơn là gửi tiết kiệm           |
| **nr.employed**        | Số lượng người có việc làm      | Việc làm nhiều hơn                 | 🔻 Giảm khả năng gửi tiền               | **Khá mạnh (âm)** | Phản ánh mối liên hệ giữa thị trường lao động và hành vi tiết kiệm         |


## ⚖️ 3️⃣ XỬ LÝ MẤT CÂN BẰNG DỮ LIỆU
Dữ liệu bị **mất cân bằng** khi tỷ lệ khách hàng gửi tiền (`y=1`) rất nhỏ so với `y=0`.  
Ta dùng **SMOTE (Synthetic Minority Oversampling Technique)** để tạo thêm mẫu thiểu số nhân tạo.  
- Ưu điểm: Giữ lại toàn bộ dữ liệu gốc, tránh mất thông tin.  
- Nhược điểm: Có thể tạo nhiễu nếu dữ liệu thiểu số không phân bố tốt.


In [52]:
from pyspark.sql import functions as F, Row, SparkSession
from pyspark.sql.types import DoubleType, StructType, StructField, IntegerType
from pyspark.sql.functions import col
import random

# --- 1️⃣ Kiểm tra tỷ lệ nhãn ---
print("=== Tỷ lệ nhãn ban đầu ===")
data.groupBy("y").count().show()

# --- 2️⃣ Phân tách dữ liệu ---
yes_df = data.filter(col("y") == "yes")
no_df  = data.filter(col("y") == "no")

yes_count = yes_df.count()
no_count  = no_df.count()

print(f"Số lượng ban đầu -> YES: {yes_count}, NO: {no_count}")

# --- 3️⃣ Nếu mất cân bằng, thực hiện oversampling ---
if yes_count < no_count:
    diff = no_count - yes_count
    yes_rows = yes_df.collect()
    new_rows = []

    for i in range(diff):
        r1, r2 = random.sample(yes_rows, 2)
        new_data = {}
        for c in data.columns:
            if c == "y":
                new_data[c] = "yes"
            else:
                try:
                    v1 = float(r1[c])
                    v2 = float(r2[c])
                    alpha = random.random()
                    new_data[c] = v1 + alpha * (v2 - v1)
                except Exception:
                    new_data[c] = r1[c]
        new_rows.append(Row(**new_data))

    # --- 🔧 Chuyển schema để chấp nhận DoubleType ---
    new_schema = StructType([
        StructField(f.name,
                    DoubleType() if isinstance(f.dataType, IntegerType) else f.dataType)
        for f in data.schema
    ])

    new_df = spark.createDataFrame(new_rows, schema=new_schema)

    # --- 🔧 Union theo tên cột (tự ép kiểu tương thích) ---
    data_balanced = data.unionByName(new_df)
else:
    data_balanced = data

# --- 4️⃣ Kiểm tra kết quả sau cân bằng ---
print("=== Dữ liệu sau oversampling (SMOTE thủ công) ===")
data_balanced.groupBy("y").count().withColumn(
    "tỷ lệ (%)", F.round(col("count") / data_balanced.count() * 100, 2)
).show()


=== Tỷ lệ nhãn ban đầu ===
+---+-----+
|  y|count|
+---+-----+
| no|36548|
|yes| 4640|
+---+-----+

Số lượng ban đầu -> YES: 4640, NO: 36548
=== Dữ liệu sau oversampling (SMOTE thủ công) ===
+---+-----+---------+
|  y|count|tỷ lệ (%)|
+---+-----+---------+
| no|36548|     50.0|
|yes|36548|     50.0|
+---+-----+---------+



## 🤖 4️⃣ XÂY DỰNG MÔ HÌNH CƠ BẢN
Ta bắt đầu bằng các **mô hình cơ bản** để thiết lập baseline:  
1. **Logistic Regression** – đơn giản, dễ giải thích, dùng làm baseline.  
2. **Random Forest** – bắt được quan hệ phi tuyến giữa các biến.  
3. **Gradient Boosted Trees (GBT)** – học sâu hơn mối quan hệ vĩ mô.  

Việc đánh giá dùng chỉ số **Accuracy, Precision, Recall, F1-score**.


In [53]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier

# Đổi tên để tránh lỗi ký tự "."
data_balanced = (data_balanced
    .withColumnRenamed("emp.var.rate", "emp_var_rate")
    .withColumnRenamed("cons.price.idx", "cons_price_idx")
    .withColumnRenamed("cons.conf.idx", "cons_conf_idx")
    .withColumnRenamed("nr.employed", "nr_employed")
)

# Vector features
assembler = VectorAssembler(
    inputCols=["emp_var_rate", "cons_price_idx", "cons_conf_idx", "euribor3m", "nr_employed"],
    outputCol="features"
)
final_data = assembler.transform(data_balanced).select("features", "y")

# Encode nhãn (PySpark yêu cầu dạng số)
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="y", outputCol="label")
final_data = indexer.fit(final_data).transform(final_data)

train_data, test_data = final_data.randomSplit([0.8, 0.2], seed=42)


In [54]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.sql.functions import col

# --- Khởi tạo mô hình ---
models = {
    "Logistic Regression": LogisticRegression(featuresCol="features", labelCol="label"),
    "Random Forest": RandomForestClassifier(featuresCol="features", labelCol="label", numTrees=50),
    "Gradient Boosted Trees": GBTClassifier(featuresCol="features", labelCol="label")
}

# --- Hàm tự tính các chỉ số ---
def compute_metrics(df):
    TP = df.filter((col("label") == 1) & (col("prediction") == 1)).count()
    FP = df.filter((col("label") == 0) & (col("prediction") == 1)).count()
    FN = df.filter((col("label") == 1) & (col("prediction") == 0)).count()
    TN = df.filter((col("label") == 0) & (col("prediction") == 0)).count()
    
    precision = TP / (TP + FP) if TP + FP > 0 else 0
    recall = TP / (TP + FN) if TP + FN > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0
    accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0
    
    return precision, recall, f1, accuracy, TP, FP, FN, TN

# --- Huấn luyện & đánh giá ---
results = []

for name, model in models.items():
    print(f"\n=== Mô hình: {name} ===")
    model_fit = model.fit(train_data)
    pred = model_fit.transform(test_data)
    
    precision, recall, f1, accuracy, TP, FP, FN, TN = compute_metrics(pred)

    # --- Hiển thị Confusion Matrix dạng bảng ---
    print(f"\nConfusion Matrix ({name}):")
    print("                 Pred=0     Pred=1")
    print(f"Actual=0     |   {TN:6d}   |   {FP:6d}")
    print(f"Actual=1     |   {FN:6d}   |   {TP:6d}")

    print(f"\nPrecision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}, Accuracy: {accuracy:.4f}")
    
    results.append((name, precision, recall, f1, accuracy, TP, FP, FN, TN))





=== Mô hình: Logistic Regression ===

Confusion Matrix (Logistic Regression):
                 Pred=0     Pred=1
Actual=0     |     5275   |     2010
Actual=1     |     1894   |     5277

Precision: 0.7242, Recall: 0.7359, F1: 0.7300, Accuracy: 0.7299

=== Mô hình: Random Forest ===

Confusion Matrix (Random Forest):
                 Pred=0     Pred=1
Actual=0     |     6691   |      594
Actual=1     |      849   |     6322

Precision: 0.9141, Recall: 0.8816, F1: 0.8976, Accuracy: 0.9002

=== Mô hình: Gradient Boosted Trees ===

Confusion Matrix (Gradient Boosted Trees):
                 Pred=0     Pred=1
Actual=0     |     6691   |      594
Actual=1     |      722   |     6449

Precision: 0.9157, Recall: 0.8993, F1: 0.9074, Accuracy: 0.9090


In [55]:
# --- Tạo bảng tổng hợp ---
print("\n=== Tổng hợp kết quả ===")
summary_df = spark.createDataFrame(
    results, 
    ["Model", "Precision", "Recall", "F1", "Accuracy", "TP", "FP", "FN", "TN"]
)
summary_df.show(truncate=False)


=== Tổng hợp kết quả ===
+----------------------+------------------+------------------+------------------+------------------+----+----+----+----+
|Model                 |Precision         |Recall            |F1                |Accuracy          |TP  |FP  |FN  |TN  |
+----------------------+------------------+------------------+------------------+------------------+----+----+----+----+
|Logistic Regression   |0.7241663235899547|0.7358806303165528|0.7299764836076912|0.7299391256225789|5277|2010|1894|5275|
|Random Forest         |0.9141122035858879|0.8816064705062056|0.897565130971818 |0.9001798561151079|6322|594 |849 |6691|
|Gradient Boosted Trees|0.9156609399403663|0.8993166922326036|0.9074152244266218|0.9089651355838406|6449|594 |722 |6691|
+----------------------+------------------+------------------+------------------+------------------+----+----+----+----+



## 📊 5️⃣ KẾT QUẢ ĐÁNH GIÁ


In [56]:
feature_cols = ["emp_var_rate", "cons_price_idx", "cons_conf_idx", "euribor3m", "nr_employed"]
importance_summary = []

for name, model in models.items():
    fitted = model.fit(train_data)
    if hasattr(fitted, "featureImportances"):
        importances = list(fitted.featureImportances)
    elif hasattr(fitted, "coefficients"):
        importances = [abs(x) for x in fitted.coefficients]
    else:
        importances = [0.0]*len(feature_cols)
    
    pairs = list(zip(feature_cols, importances))
    print(f"\n=== Độ quan trọng ({name}) ===")
    for col_name, imp in pairs:
        print(f"{col_name:<20}: {imp:.6f}")
    
    max_idx = importances.index(max(importances, key=abs))
    importance_summary.append((name, feature_cols[max_idx], importances[max_idx]))

# Tổng kết
print("\n=== Tổng kết biến quan trọng nhất ===")
print(f"{'Mô hình':<30} {'Biến quan trọng nhất':<25} {'Giá trị':>10}")
for row in importance_summary:
    print(f"{row[0]:<30} {row[1]:<25} {row[2]:>10.6f}")



=== Độ quan trọng (Logistic Regression) ===
emp_var_rate        : 0.273858
cons_price_idx      : 0.532919
cons_conf_idx       : 0.061193
euribor3m           : 0.216819
nr_employed         : 0.008131

=== Độ quan trọng (Random Forest) ===
emp_var_rate        : 0.076077
cons_price_idx      : 0.046925
cons_conf_idx       : 0.256145
euribor3m           : 0.237378
nr_employed         : 0.383475

=== Độ quan trọng (Gradient Boosted Trees) ===
emp_var_rate        : 0.239829
cons_price_idx      : 0.125229
cons_conf_idx       : 0.293558
euribor3m           : 0.073672
nr_employed         : 0.267712

=== Tổng kết biến quan trọng nhất ===
Mô hình                        Biến quan trọng nhất         Giá trị
Logistic Regression            cons_price_idx              0.532919
Random Forest                  nr_employed                 0.383475
Gradient Boosted Trees         cons_conf_idx               0.293558


##  Phân tích theo thời gian và biến kinh tế vĩ mô

Mục tiêu của phần này là mở rộng EDA bằng cách xem **xu hướng thời gian** ảnh hưởng thế nào đến khả năng khách hàng đăng ký gửi tiền (`y = yes`).  
Các bước chính:
1. Tạo thêm các biến thời gian (tháng số, mùa).  
2. Phân tích tỷ lệ đăng ký (`conversion rate`) theo **tháng** và **mùa**.  
3. Kết hợp các biến kinh tế vĩ mô để tìm hiểu **trong thời điểm tỷ lệ gửi tiền cao, các biến kinh tế vĩ mô có đặc điểm gì**.  


In [57]:
# --- 1️⃣ Tạo thêm các biến thời gian (tháng số, mùa) ---

from pyspark.sql.functions import when, col

# Map tháng sang số
month_mapping = {
    "jan": 1, "feb": 2, "mar": 3, "apr": 4,
    "may": 5, "jun": 6, "jul": 7, "aug": 8,
    "sep": 9, "oct": 10, "nov": 11, "dec": 12
}

# Biến tháng dạng số
month_expr = None
for m, n in month_mapping.items():
    expr = when(col("month") == m, n)
    month_expr = expr if month_expr is None else month_expr.when(col("month") == m, n)
month_expr = month_expr.otherwise(None)

data = data.withColumn("month_num", month_expr)

# Tạo biến mùa (xuân, hạ, thu, đông)
data = data.withColumn(
    "season",
    when(col("month_num").isin(3,4,5), "spring")
    .when(col("month_num").isin(6,7,8), "summer")
    .when(col("month_num").isin(9,10,11), "autumn")
    .otherwise("winter")
)

print("✅ Đã thêm biến month_num và season")
data.select("month", "month_num", "season").distinct().orderBy("month_num").show()


✅ Đã thêm biến month_num và season
+-----+---------+------+
|month|month_num|season|
+-----+---------+------+
|  0.0|     NULL|winter|
+-----+---------+------+



###  Phân tích tỷ lệ đăng ký theo thời gian
Xem xét phân bố biến mục tiêu `y` theo **tháng** và **mùa** để xác định giai đoạn nào có nhiều khách hàng gửi tiền nhất.


In [58]:
from pyspark.sql.functions import count, sum

# --- 2️⃣ Tỷ lệ đăng ký theo tháng ---
print("=== Tỷ lệ khách hàng đăng ký theo tháng ===")
conv_by_month = (
    data.groupBy("month")
    .agg(
        count("*").alias("total"),
        sum(when(col("y") == "yes", 1).otherwise(0)).alias("yes_count")
    )
    .withColumn("conversion_rate", col("yes_count") / col("total") * 100)
    .orderBy("month")
)
conv_by_month.show(truncate=False)

# --- 3️⃣ Tỷ lệ đăng ký theo mùa ---
print("=== Tỷ lệ khách hàng đăng ký theo mùa ===")
conv_by_season = (
    data.groupBy("season")
    .agg(
        count("*").alias("total"),
        sum(when(col("y") == "yes", 1).otherwise(0)).alias("yes_count")
    )
    .withColumn("conversion_rate", col("yes_count") / col("total") * 100)
    .orderBy("season")
)
conv_by_season.show(truncate=False)


=== Tỷ lệ khách hàng đăng ký theo tháng ===
+-----+-----+---------+------------------+
|month|total|yes_count|conversion_rate   |
+-----+-----+---------+------------------+
|0.0  |41188|4640     |11.265417111780131|
+-----+-----+---------+------------------+

=== Tỷ lệ khách hàng đăng ký theo mùa ===
+------+-----+---------+------------------+
|season|total|yes_count|conversion_rate   |
+------+-----+---------+------------------+
|winter|41188|4640     |11.265417111780131|
+------+-----+---------+------------------+



###  Liên hệ giữa tỷ lệ đăng ký và các biến kinh tế vĩ mô

Bước này giúp hiểu rõ **khi tỷ lệ gửi tiền cao**, các yếu tố như **lãi suất, việc làm, lạm phát, niềm tin tiêu dùng** có giá trị như thế nào.


In [59]:
from pyspark.sql.functions import mean

macro_cols = ["emp_var_rate", "cons_price_idx", "cons_conf_idx", "euribor3m", "nr_employed"]

# --- 4️⃣ Trung bình biến vĩ mô theo từng tháng ---
macro_by_month = data.groupBy("month").agg(
    *[mean(c).alias(f"avg_{c}") for c in macro_cols]
)

# --- 5️⃣ Kết hợp với conversion rate ---
result = conv_by_month.join(macro_by_month, on="month", how="inner")

# --- 6️⃣ Hiển thị kết quả theo thứ tự conversion giảm dần ---
print("=== So sánh tỷ lệ đăng ký và đặc trưng kinh tế từng tháng ===")
result.orderBy(col("conversion_rate").desc()).show(truncate=False)


=== So sánh tỷ lệ đăng ký và đặc trưng kinh tế từng tháng ===
+-----+-----+---------+------------------+-------------------+------------------+------------------+------------------+-----------------+
|month|total|yes_count|conversion_rate   |avg_emp_var_rate   |avg_cons_price_idx|avg_cons_conf_idx |avg_euribor3m     |avg_nr_employed  |
+-----+-----+---------+------------------+-------------------+------------------+------------------+------------------+-----------------+
|0.0  |41188|4640     |11.265417111780131|0.08188550063178392|93.57566436828918 |-40.50260027191787|3.6212908128585366|5167.035910944004|
+-----+-----+---------+------------------+-------------------+------------------+------------------+------------------+-----------------+



In [60]:
from pyspark.sql.functions import col, when, lit
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

# --- Thêm tháng & mùa ---
data_balanced = data_balanced.withColumn("month", col("month").cast("int"))

data_balanced = data_balanced.withColumn(
    "season",
    when(col("month").isin(12, 1, 2), lit("Winter"))
    .when(col("month").isin(3, 4, 5), lit("Spring"))
    .when(col("month").isin(6, 7, 8), lit("Summer"))
    .when(col("month").isin(9, 10, 11), lit("Autumn"))
    .otherwise(lit("Unknown"))
)

# --- Đảm bảo không có null ---
data_balanced = data_balanced.na.fill({
    "emp_var_rate": 0.0,
    "cons_price_idx": 0.0,
    "cons_conf_idx": 0.0,
    "euribor3m": 0.0,
    "nr_employed": 0.0,
    "month": 0,
    "season": "Unknown"
})

# --- Encode cột season ---
season_indexer = StringIndexer(
    inputCol="season", outputCol="season_index", handleInvalid="keep"
)
data_encoded = season_indexer.fit(data_balanced).transform(data_balanced)

encoder = OneHotEncoder(inputCols=["season_index"], outputCols=["season_vec"])
data_encoded = encoder.fit(data_encoded).transform(data_encoded)

# --- Ép kiểu tất cả cột numeric ---
numeric_cols = ["emp_var_rate", "cons_price_idx", "cons_conf_idx", "euribor3m", "nr_employed", "month"]
for col_name in numeric_cols:
    data_encoded = data_encoded.withColumn(col_name, col(col_name).cast("double"))

# --- Assemble features ---
feature_cols = numeric_cols + ["season_vec"]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

final_data = assembler.transform(data_encoded).select("features", "y")

# --- Encode nhãn ---
indexer = StringIndexer(inputCol="y", outputCol="label")
final_data = indexer.fit(final_data).transform(final_data)

# --- Tách tập train/test ---
train_data, test_data = final_data.randomSplit([0.8, 0.2], seed=42)

print("✅ Hoàn tất xử lý dữ liệu — tất cả biến đã numeric và không null.")
final_data.show(3, truncate=False)


✅ Hoàn tất xử lý dữ liệu — tất cả biến đã numeric và không null.
+---------------------------------------+---+-----+
|features                               |y  |label|
+---------------------------------------+---+-----+
|[1.1,93.994,-36.4,4.857,5191.0,0.0,1.0]|no |0.0  |
|[1.1,93.994,-36.4,4.857,5191.0,0.0,1.0]|no |0.0  |
|[1.1,93.994,-36.4,4.857,5191.0,0.0,1.0]|no |0.0  |
+---------------------------------------+---+-----+
only showing top 3 rows



##  Huấn luyện mô hình với Cross Validation
Áp dụng Logistic Regression, Random Forest và Gradient Boosted Trees — thêm tuning siêu tham số.


In [61]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import os

spark = SparkSession.builder.getOrCreate()

# --- Khởi tạo mô hình ---
models = {
    "Logistic Regression": LogisticRegression(featuresCol="features", labelCol="label"),
    "Random Forest": RandomForestClassifier(featuresCol="features", labelCol="label"),
    "Gradient Boosted Trees": GBTClassifier(featuresCol="features", labelCol="label")
}

evaluator = BinaryClassificationEvaluator(labelCol="label", metricName="areaUnderROC")

# --- Hàm tính chỉ số ---
def compute_metrics(df):
    TP = df.filter((col("label") == 1) & (col("prediction") == 1)).count()
    FP = df.filter((col("label") == 0) & (col("prediction") == 1)).count()
    FN = df.filter((col("label") == 1) & (col("prediction") == 0)).count()
    TN = df.filter((col("label") == 0) & (col("prediction") == 0)).count()

    precision = TP / (TP + FP) if TP + FP > 0 else 0
    recall = TP / (TP + FN) if TP + FN > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0
    accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0

    return TP, FP, FN, TN, precision, recall, f1, accuracy


# --- Thư mục lưu model ---
os.makedirs("saved_models", exist_ok=True)

cv_results = {}
best_models = {}

# --- Train tất cả mô hình ---
for name, model in models.items():
    print(f"\n🚀 Training model: {name}")

    # --- Lưới tham số ---
    if isinstance(model, LogisticRegression):
        grid = ParamGridBuilder() \
            .addGrid(model.regParam, [0.01, 0.1, 1.0]) \
            .addGrid(model.elasticNetParam, [0.0, 0.5, 1.0]) \
            .build()
    elif isinstance(model, RandomForestClassifier):
        grid = ParamGridBuilder() \
            .addGrid(model.numTrees, [30, 50, 100]) \
            .addGrid(model.maxDepth, [5, 10]) \
            .build()
    elif isinstance(model, GBTClassifier):
        grid = ParamGridBuilder() \
            .addGrid(model.maxDepth, [3, 5]) \
            .addGrid(model.maxIter, [20, 40]) \
            .build()
    else:
        grid = ParamGridBuilder().build()

    cv = CrossValidator(
        estimator=model,
        estimatorParamMaps=grid,
        evaluator=evaluator,
        numFolds=3,
        parallelism=2
    )

    # --- Train và dự đoán ---
    cv_model = cv.fit(train_data)
    preds = cv_model.transform(test_data)

    auc_per_param = list(cv_model.avgMetrics)
    auc_best = evaluator.evaluate(preds)
    TP, FP, FN, TN, precision, recall, f1, accuracy = compute_metrics(preds)

    # --- Lưu kết quả ---
    cv_results[name] = {
        "model": cv_model.bestModel,
        "preds": preds,
        "auc_best": auc_best,
        "auc_list": auc_per_param,
        "metrics": {
            "TP": TP, "FP": FP, "FN": FN, "TN": TN,
            "precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy
        }
    }

    best_models[name] = cv_model.bestModel

    # --- In confusion matrix và metric ---
    print(f"\nConfusion Matrix ({name}):")
    print("                 Pred=0     Pred=1")
    print(f"Actual=0     |   {TN:6d}   |   {FP:6d}")
    print(f"Actual=1     |   {FN:6d}   |   {TP:6d}")
    print(f"\nPrecision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}, Accuracy: {accuracy:.4f}, AUC: {auc_best:.4f}")


print("\n✅ Đã train và lưu xong tất cả mô hình. Biến `best_models` và `cv_results` đã sẵn sàng.")



🚀 Training model: Logistic Regression

Confusion Matrix (Logistic Regression):
                 Pred=0     Pred=1
Actual=0     |     5275   |     2010
Actual=1     |     1883   |     5288

Precision: 0.7246, Recall: 0.7374, F1: 0.7309, Accuracy: 0.7307, AUC: 0.8048

🚀 Training model: Random Forest

Confusion Matrix (Random Forest):
                 Pred=0     Pred=1
Actual=0     |     6874   |      411
Actual=1     |      813   |     6358

Precision: 0.9393, Recall: 0.8866, F1: 0.9122, Accuracy: 0.9153, AUC: 0.9610

🚀 Training model: Gradient Boosted Trees

Confusion Matrix (Gradient Boosted Trees):
                 Pred=0     Pred=1
Actual=0     |     6796   |      489
Actual=1     |      748   |     6423

Precision: 0.9293, Recall: 0.8957, F1: 0.9122, Accuracy: 0.9144, AUC: 0.9628

✅ Đã train và lưu xong tất cả mô hình. Biến `best_models` và `cv_results` đã sẵn sàng.


###  Kết quả tổng hợp


In [62]:
print("=== Quá trình tuning AUC từng mô hình ===\n")

for name, info in cv_results.items():
    print(f"\n👉 {name}")
    auc_list = info["auc_list"]
    for i, auc_val in enumerate(auc_list):
        print(f"  - Lần {i+1}: AUC = {auc_val:.4f}")
    print(f"✅ Best AUC = {info['auc_best']:.4f}")


=== Quá trình tuning AUC từng mô hình ===


👉 Logistic Regression
  - Lần 1: AUC = 0.8028
  - Lần 2: AUC = 0.8013
  - Lần 3: AUC = 0.8003
  - Lần 4: AUC = 0.7986
  - Lần 5: AUC = 0.7971
  - Lần 6: AUC = 0.7987
  - Lần 7: AUC = 0.7908
  - Lần 8: AUC = 0.5000
  - Lần 9: AUC = 0.5000
✅ Best AUC = 0.8048

👉 Random Forest
  - Lần 1: AUC = 0.9368
  - Lần 2: AUC = 0.9599
  - Lần 3: AUC = 0.9379
  - Lần 4: AUC = 0.9593
  - Lần 5: AUC = 0.9384
  - Lần 6: AUC = 0.9595
✅ Best AUC = 0.9610

👉 Gradient Boosted Trees
  - Lần 1: AUC = 0.9503
  - Lần 2: AUC = 0.9568
  - Lần 3: AUC = 0.9605
  - Lần 4: AUC = 0.9614
✅ Best AUC = 0.9628


In [63]:
print("=== Confusion Matrix từng mô hình ===\n")

for name, info in cv_results.items():
    m = info["metrics"]
    print(f"\n📘 {name}")
    print("                 Pred=0     Pred=1")
    print(f"Actual=0     |   {m['TN']:6d}   |   {m['FP']:6d}")
    print(f"Actual=1     |   {m['FN']:6d}   |   {m['TP']:6d}")
    print(f"\nPrecision: {m['precision']:.4f}, Recall: {m['recall']:.4f}, F1: {m['f1']:.4f}, Accuracy: {m['accuracy']:.4f}")


=== Confusion Matrix từng mô hình ===


📘 Logistic Regression
                 Pred=0     Pred=1
Actual=0     |     5275   |     2010
Actual=1     |     1883   |     5288

Precision: 0.7246, Recall: 0.7374, F1: 0.7309, Accuracy: 0.7307

📘 Random Forest
                 Pred=0     Pred=1
Actual=0     |     6874   |      411
Actual=1     |      813   |     6358

Precision: 0.9393, Recall: 0.8866, F1: 0.9122, Accuracy: 0.9153

📘 Gradient Boosted Trees
                 Pred=0     Pred=1
Actual=0     |     6796   |      489
Actual=1     |      748   |     6423

Precision: 0.9293, Recall: 0.8957, F1: 0.9122, Accuracy: 0.9144


In [64]:
print("=== Tổng hợp kết quả tất cả mô hình ===\n")
print(f"{'Model':25s} | {'AUC':6s} | {'Precision':10s} | {'Recall':8s} | {'F1':6s} | {'Acc':6s}")
print("-"*70)
for name, info in cv_results.items():
    m = info["metrics"]
    print(f"{name:25s} | {info['auc_best']:.4f} | {m['precision']:.4f}     | {m['recall']:.4f}  | {m['f1']:.4f} | {m['accuracy']:.4f}")


=== Tổng hợp kết quả tất cả mô hình ===

Model                     | AUC    | Precision  | Recall   | F1     | Acc   
----------------------------------------------------------------------
Logistic Regression       | 0.8048 | 0.7246     | 0.7374  | 0.7309 | 0.7307
Random Forest             | 0.9610 | 0.9393     | 0.8866  | 0.9122 | 0.9153
Gradient Boosted Trees    | 0.9628 | 0.9293     | 0.8957  | 0.9122 | 0.9144


###  Phân tích tầm quan trọng của biến sau khi thêm "tháng" và "mùa"


In [65]:
from pyspark.sql import Row
from pyspark.sql.functions import col

importance_rows = []

for name, fitted in best_models.items():
    if hasattr(fitted, "featureImportances"):  # RandomForest / GBT
        importances = list(fitted.featureImportances)
        for feat, imp in zip(feature_cols, importances):
            importance_rows.append(Row(model=name, feature=feat, importance=float(imp)))

    elif hasattr(fitted, "coefficients"):  # Logistic Regression
        for feat, imp in zip(feature_cols, fitted.coefficients):
            importance_rows.append(Row(model=name, feature=feat, importance=float(abs(imp))))

    else:
        print(f"⚠️ {name}: Không hỗ trợ trích xuất feature importance.")

# --- Tạo DataFrame Spark ---
feature_importance_df = spark.createDataFrame(importance_rows)
print("✅ Đã tạo feature_importance_df (chứa tất cả mô hình).")


✅ Đã tạo feature_importance_df (chứa tất cả mô hình).


In [66]:
print("📊 Feature Importance: Logistic Regression")
feature_importance_df.filter(col("model") == "Logistic Regression") \
    .orderBy(col("importance").desc()) \
    .show(truncate=False)


📊 Feature Importance: Logistic Regression
+-------------------+--------------+--------------------+
|model              |feature       |importance          |
+-------------------+--------------+--------------------+
|Logistic Regression|cons_price_idx|0.431698656457129   |
|Logistic Regression|emp_var_rate  |0.24618010540976207 |
|Logistic Regression|euribor3m     |0.21827262608142817 |
|Logistic Regression|cons_conf_idx |0.05621535092980314 |
|Logistic Regression|nr_employed   |0.007634176901973183|
|Logistic Regression|month         |0.0                 |
|Logistic Regression|season_vec    |0.0                 |
+-------------------+--------------+--------------------+



In [67]:
print("📊 Feature Importance: Random Forest")
feature_importance_df.filter(col("model") == "Random Forest") \
    .orderBy(col("importance").desc()) \
    .show(truncate=False)


📊 Feature Importance: Random Forest
+-------------+--------------+-------------------+
|model        |feature       |importance         |
+-------------+--------------+-------------------+
|Random Forest|nr_employed   |0.2632992099402898 |
|Random Forest|euribor3m     |0.24891744827229528|
|Random Forest|cons_conf_idx |0.19121900709587356|
|Random Forest|emp_var_rate  |0.18997014400303414|
|Random Forest|cons_price_idx|0.1065941906885072 |
|Random Forest|month         |0.0                |
|Random Forest|season_vec    |0.0                |
+-------------+--------------+-------------------+



In [68]:
print("📊 Feature Importance: Gradient Boosted Trees")
feature_importance_df.filter(col("model") == "Gradient Boosted Trees") \
    .orderBy(col("importance").desc()) \
    .show(truncate=False)


📊 Feature Importance: Gradient Boosted Trees
+----------------------+--------------+-------------------+
|model                 |feature       |importance         |
+----------------------+--------------+-------------------+
|Gradient Boosted Trees|cons_conf_idx |0.2805864919125758 |
|Gradient Boosted Trees|nr_employed   |0.26725407529419015|
|Gradient Boosted Trees|emp_var_rate  |0.23827387194195473|
|Gradient Boosted Trees|cons_price_idx|0.13602102954187395|
|Gradient Boosted Trees|euribor3m     |0.07786453130940538|
|Gradient Boosted Trees|month         |0.0                |
|Gradient Boosted Trees|season_vec    |0.0                |
+----------------------+--------------+-------------------+

